In [1]:
#Copyright 2020 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn import svm

import pandas as pd
import numpy as np
np.random.seed(512)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np 
import pandas as pd

In [2]:
def ProcessStats(data,y):

    data1 = data[['total_vals', 'num_nans', '%_nans', 'num_of_dist_val', '%_dist_val', 'mean',
        'std_dev', 'min_val', 'max_val','has_delimiters', 'has_url', 'has_email', 'has_date', 'mean_word_count',
       'std_dev_word_count', 'mean_stopword_total', 'stdev_stopword_total',
       'mean_char_count', 'stdev_char_count', 'mean_whitespace_count',
       'stdev_whitespace_count', 'mean_delim_count', 'stdev_delim_count',
       'is_list', 'is_long_sentence']]
    data1 = data1.reset_index(drop=True)
    data1 = data1.fillna(0)    

    def abs_limit(x):
        if abs(x) > 10000:
            return 10000*np.sign(x)
        return x

    column_names_to_normalize = ['total_vals', 'num_nans', '%_nans', 'num_of_dist_val', '%_dist_val', 'mean',
        'std_dev', 'min_val', 'max_val','has_delimiters', 'has_url', 'has_email', 'has_date', 'mean_word_count',
       'std_dev_word_count', 'mean_stopword_total', 'stdev_stopword_total',
       'mean_char_count', 'stdev_char_count', 'mean_whitespace_count',
       'stdev_whitespace_count', 'mean_delim_count', 'stdev_delim_count',
       'is_list', 'is_long_sentence']
    
    for col in column_names_to_normalize:
        data1[col] = data1[col].apply(abs_limit)
    
    print(column_names_to_normalize)
    x = data1[column_names_to_normalize].values
    x = np.nan_to_num(x)
    x_scaled = StandardScaler().fit_transform(x)
    df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index=data1.index)
    data1[column_names_to_normalize] = df_temp


    y.y_act = y.y_act.astype(float)

    print(f"> Data mean: {data1.mean()}\n")
    print(f"> Data median: {data1.median()}\n")
    print(f"> Data stdev: {data1.std()}")
    
    return data1

vectorizerName = CountVectorizer(ngram_range=(2, 2), analyzer='char')
vectorizerSample = CountVectorizer(ngram_range=(2, 2), analyzer='char')


def FeatureExtraction(data,data1,flag):

    arr = data['Attribute_name'].values
    arr = [str(x) for x in arr]

    arr1 = data['sample_1'].values
    arr1 = [str(x) for x in arr1]
    arr2 = data['sample_2'].values
    arr2 = [str(x) for x in arr2]
    arr3 = data['sample_3'].values
    arr3 = [str(x) for x in arr3]    
    print(len(arr1),len(arr2))
    if flag:
        X = vectorizerName.fit_transform(arr)
        X1 = vectorizerSample.fit_transform(arr1)
        X2 = vectorizerSample.transform(arr2)   
        
    else:
        X = vectorizerName.transform(arr)
        X1 = vectorizerSample.transform(arr1)
        X2 = vectorizerSample.transform(arr2)        
        
#     print(f"> Length of vectorized feature_names: {len(vectorizer.get_feature_names())}")

    attr_df = pd.DataFrame(X.toarray())
    sample1_df = pd.DataFrame(X1.toarray())
    sample2_df = pd.DataFrame(X2.toarray())
    print(len(data1),len(attr_df),len(sample1_df),len(sample2_df))   
    
    data2 = pd.concat([data1, attr_df], axis=1, sort=False)
    print(len(data2))
    return data2

In [3]:
xtrain = pd.read_csv('../../Benchmark-Labeled-Data/data_train.csv')
xtest = pd.read_csv('../../Benchmark-Labeled-Data/data_test.csv')


xtrain = xtrain.sample(frac=1,random_state=100).reset_index(drop=True)
print(len(xtrain))

y_train = xtrain.loc[:,['y_act']]
y_test = xtest.loc[:,['y_act']]

7936


In [4]:
dict_label = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

y_train['y_act'] = [dict_label[i] for i in y_train['y_act']]
y_test['y_act'] = [dict_label[i] for i in y_test['y_act']]
y_train

,y_act
0,0
1,0
2,0
3,1
4,0
...,...
7931,0
7932,1
7933,8
7934,7


In [5]:
xtrain1 = ProcessStats(xtrain,y_train)
xtest1 = ProcessStats(xtest,y_test)


X_train = FeatureExtraction(xtrain,xtrain1,1)
X_test = FeatureExtraction(xtest,xtest1,0)


X_train_new = X_train.reset_index(drop=True)
y_train_new = y_train.reset_index(drop=True)
X_train_new = X_train_new.values
y_train_new = y_train_new.values

k = 5
kf = KFold(n_splits=k)
avg_train_acc,avg_test_acc = 0,0
    
cvals = [0.1,1,10,100,1000]
gamavals = [0.0001,0.001,0.01,0.1,1,10]

# cvals = [1]
# gamavals = [0.0001]

bestPerformingModel = svm.SVC(C=100,decision_function_shape="ovo", gamma = 0.001, cache_size=20000, kernel="sigmoid")
bestscore = 0


avgsc_lst,avgsc_train_lst,avgsc_hld_lst = [],[],[]
avgsc,avgsc_train,avgsc_hld = 0,0,0

for train_index, test_index in kf.split(X_train_new):
    X_train_cur, X_test_cur = X_train_new[train_index], X_train_new[test_index]
    y_train_cur, y_test_cur = y_train_new[train_index], y_train_new[test_index]
    X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=100)

    bestPerformingModel = svm.SVC(C=100,decision_function_shape="ovo", gamma = 0.001, cache_size=20000, kernel="sigmoid")
    bestscore = 0
    for cval in cvals:
        for gval in gamavals:
            clf = svm.SVC(C = cval,decision_function_shape="ovo", gamma = gval, cache_size=20000, kernel="sigmoid")
            clf.fit(X_train_train, y_train_train)
            sc = clf.score(X_val, y_val)

            if bestscore < sc:
                bestscore = sc
                bestPerformingModel = clf
#                 print(bestPerformingModel)

    bscr_train = bestPerformingModel.score(X_train_cur, y_train_cur)
    bscr = bestPerformingModel.score(X_test_cur, y_test_cur)
    bscr_hld = bestPerformingModel.score(X_test, y_test)

    avgsc_train_lst.append(bscr_train)
    avgsc_lst.append(bscr)
    avgsc_hld_lst.append(bscr_hld)
    
    avgsc_train = avgsc_train + bscr_train    
    avgsc = avgsc + bscr
    avgsc_hld = avgsc_hld + bscr_hld

    print(bscr_train)
    print(bscr)
    print(bscr_hld)

['total_vals', 'num_nans', '%_nans', 'num_of_dist_val', '%_dist_val', 'mean', 'std_dev', 'min_val', 'max_val', 'has_delimiters', 'has_url', 'has_email', 'has_date', 'mean_word_count', 'std_dev_word_count', 'mean_stopword_total', 'stdev_stopword_total', 'mean_char_count', 'stdev_char_count', 'mean_whitespace_count', 'stdev_whitespace_count', 'mean_delim_count', 'stdev_delim_count', 'is_list', 'is_long_sentence']
> Data mean: total_vals               -3.939501e-17
num_nans                 -3.044160e-17
%_nans                   -5.260129e-17
num_of_dist_val          -2.865092e-17
%_dist_val                7.341797e-17
mean                     -4.387172e-17
std_dev                  -2.686023e-18
min_val                  -4.029035e-17
max_val                   9.848753e-18
has_delimiters           -3.581365e-17
has_url                   2.686023e-17
has_email                -9.401082e-18
has_date                  3.984268e-17
mean_word_count           1.096793e-17
std_dev_word_count       -

/home/adriana/Documents/ML-Data-Prep-Zoo/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/adriana/Documents/ML-Data-Prep-Zoo/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/adriana/Documents/ML-Data-Prep-Zoo/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/adriana/Documents/ML-Data-Prep-Zoo/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:114

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [ ]:
print(avgsc_train_lst)
print(avgsc_lst)
print(avgsc_hld_lst)

print(avgsc_train/k)
print(avgsc/k)
print(avgsc_hld/k)

y_pred = bestPerformingModel.predict(X_test.to_numpy())
bscr_hld = bestPerformingModel.score(X_test, y_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print('Confusion Matrix: Actual (Row) vs Predicted (Column)')
print(cnf_matrix)

In [6]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, accuracy_score


def PrintMetrics(y_true, y_pred):
    print(f'Accuracy: {accuracy_score(y_true, y_pred)}')

    matrix = confusion_matrix(y_true, y_pred)
    print(matrix)

    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 score: {f1}')


PrintMetrics(y_test, bestPerformingModel.predict(X_test.to_numpy()))

Accuracy: 0.8277078085642318
[[642  44   1   0   0   3   0   3  14]
 [ 22 414   1   1   0   0   0   7  12]
 [  1   0 136   0   0   3   0   1   0]
 [  2   4   0  69   0   5   0  10   2]
 [  2   0   0   0  29   0   1   0   0]
 [  5   4   0   0   0  90   0   0   0]
 [  1   0   0   1   0  20  32   0   3]
 [ 25  44   2   4   1   0   0 135   4]
 [ 27  44   0   1   0   0   1  16  96]]
Precision: [0.88308116 0.74729242 0.97142857 0.90789474 0.96666667 0.74380165
 0.94117647 0.78488372 0.73282443]
Recall: [0.90806223 0.9059081  0.96453901 0.75       0.90625    0.90909091
 0.56140351 0.62790698 0.51891892]
F1 score: [0.89539749 0.8189911  0.96797153 0.82142857 0.93548387 0.81818182
 0.7032967  0.69767442 0.60759494]


// sigmoid

[[642  44   1   0   0   3   0   3  14]
 [ 22 414   1   1   0   0   0   7  12]
 [  1   0 136   0   0   3   0   1   0]
 [  2   4   0  69   0   5   0  10   2]
 [  2   0   0   0  29   0   1   0   0]
 [  5   4   0   0   0  90   0   0   0]
 [  1   0   0   1   0  20  32   0   3]
 [ 25  44   2   4   1   0   0 135   4]
 [ 27  44   0   1   0   0   1  16  96]]
Precision: [0.88308116 0.74729242 0.97142857 0.90789474 0.96666667 0.74380165
 0.94117647 0.78488372 0.73282443]
Recall: [0.90806223 0.9059081  0.96453901 0.75       0.90625    0.90909091
 0.56140351 0.62790698 0.51891892]
F1 score: [0.89539749 0.8189911  0.96797153 0.82142857 0.93548387 0.81818182
 0.7032967  0.69767442 0.60759494]

// rbf

Accuracy: 0.8624685138539043
[[663  33   1   0   0   0   0   0  10]
 [ 14 421   0   1   0   0   0   7  14]
 [  1   0 136   0   0   3   0   1   0]
 [  1   2   0  67   0   3   1  18   0]
 [  1   0   0   0  30   0   1   0   0]
 [  6   3   0   0   0  90   0   0   0]
 [  1   0   0   1   0  19  31   2   3]
 [ 12  35   2   3   1   0   0 158   4]
 [ 24  32   0   2   0   0   1  10 116]]
Precision: [0.91701245 0.80038023 0.97841727 0.90540541 0.96774194 0.7826087
 0.91176471 0.80612245 0.78911565]
Recall: [0.93776521 0.92122538 0.96453901 0.72826087 0.9375     0.90909091
 0.54385965 0.73488372 0.62702703]
F1 score: [0.92727273 0.85656155 0.97142857 0.80722892 0.95238095 0.8411215
 0.68131868 0.76885645 0.69879518]